# Assignment Module 2: Product Classification

The goal of this assignment is to implement a neural network that classifies smartphone pictures of products found in grocery stores. The assignment will be divided into two parts: first, you will be asked to implement from scratch your own neural network for image classification; then, you will fine-tune a pretrained network provided by PyTorch.


## Preliminaries: the dataset

The dataset you will be using contains natural images of products taken with a smartphone camera in different grocery stores:

<p align="center">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Granny-Smith.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Pink-Lady.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Lemon.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Banana.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Vine-Tomato.jpg" width="150">
</p>
<p align="center">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Yellow-Onion.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Green-Bell-Pepper.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Arla-Standard-Milk.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Oatly-Natural-Oatghurt.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Alpro-Fresh-Soy-Milk.jpg" width="150">
</p>

The products belong to the following 43 classes:
```
0.  Apple
1.  Avocado
2.  Banana
3.  Kiwi
4.  Lemon
5.  Lime
6.  Mango
7.  Melon
8.  Nectarine
9.  Orange
10. Papaya
11. Passion-Fruit
12. Peach
13. Pear
14. Pineapple
15. Plum
16. Pomegranate
17. Red-Grapefruit
18. Satsumas
19. Juice
20. Milk
21. Oatghurt
22. Oat-Milk
23. Sour-Cream
24. Sour-Milk
25. Soyghurt
26. Soy-Milk
27. Yoghurt
28. Asparagus
29. Aubergine
30. Cabbage
31. Carrots
32. Cucumber
33. Garlic
34. Ginger
35. Leek
36. Mushroom
37. Onion
38. Pepper
39. Potato
40. Red-Beet
41. Tomato
42. Zucchini
```

The dataset is split into training (`train`), validation (`val`), and test (`test`) set.

The following code cells download the dataset and define a `torch.utils.data.Dataset` class to access it. This `Dataset` class will be the starting point of your assignment: use it in your own code and build everything else around it.

In [6]:
!git clone https://github.com/marcusklasson/GroceryStoreDataset.git

Cloning into 'GroceryStoreDataset'...
remote: Enumerating objects: 6559, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 6559 (delta 45), reused 35 (delta 35), pack-reused 6293
Receiving objects: 100% (6559/6559), 116.26 MiB | 15.22 MiB/s, done.
Resolving deltas: 100% (275/275), done.


In [7]:
from pathlib import Path
from PIL import Image
from typing import List, Tuple

import torch
from torch import Tensor,device
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam, SGD, AdamW
import torch.nn.functional as F
import torch.nn as nn
import torchvision

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np

import random
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device: {device}")

Device: cuda


In [8]:
def fix_random(seed: int) -> None:
    """Fix all the possible sources of randomness.

    Args:
        seed: the seed to use.
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_random(45)

In [9]:
class GroceryStoreDataset(Dataset):

    def __init__(self, split: str, transform=None) -> None:
        super().__init__()

        self.root = Path("GroceryStoreDataset/dataset")
        self.split = split
        self.paths, self.labels = self.read_file()

        self.transform = transform

    def __len__(self) -> int:
        return len(self.labels)

    def __getitem__(self, idx) -> Tuple[Tensor, int]:
        img = Image.open(self.root / self.paths[idx])
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

    def read_file(self) -> Tuple[List[str], List[int]]:
        paths = []
        labels = []

        with open(self.root / f"{self.split}.txt") as f:
            for line in f:
                # path, fine-grained class, coarse-grained class
                path, _, label = line.replace("\n", "").split(", ")
                paths.append(path), labels.append(int(label))

        return paths, labels

    def get_num_classes(self) -> int:
        return max(self.labels) + 1

In [10]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])
train_dataset = GroceryStoreDataset(split='train',transform=transform)
val_dataset = GroceryStoreDataset(split='val',transform=transform)
test_dataset = GroceryStoreDataset(split='test',transform=transform)
train_dataset.__len__(),val_dataset.__len__(),test_dataset.__len__()

(2640, 296, 2485)

In [11]:
def check_image_shapes(dataset):
    for i in range(dataset.__len__()):
        img, label = dataset[i]
        w, h = 0,0
        #print(f"Image {i} shape: {img.shape} (width, height)")
        if img.shape[1] > w:
          w = img.shape[1]
        if img.shape[2] > h:
          h = img.shape[2]
    return w,h
# Check shapes of the first few images
w_t,h_t = check_image_shapes(train_dataset)
w_t,h_t
#w_v,h_v = check_image_shapes(val_dataset)
#w_T,h_T = check_image_shapes(test_dataset)

(464, 348)

In [12]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
image_height, image_width, image_channels = 348,348,3
# Define the transform to resize images
resize_transform = transforms.Compose([
    transforms.Resize((image_height, image_width)),
    transforms.ToTensor()
])

# Load the training dataset with the resizing transform
train_dataset = GroceryStoreDataset(split='train',transform=resize_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

# Initialize variables to store the mean and std
mean = torch.zeros(3)
std = torch.zeros(3)
nb_samples = 0

# Compute mean and standard deviation
for images, _ in train_loader:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

print(f'Mean: {mean}')
print(f'Std: {std}')


Mean: tensor([0.5306, 0.3964, 0.2564])
Std: tensor([0.2325, 0.2093, 0.1781])


## Part 1: design your own network

Your goal is to implement a convolutional neural network for image classification and train it on `GroceryStoreDataset`. You should consider yourselves satisfied once you obtain a classification accuracy on the **validation** split of **around 60%**. You are free to achieve that however you want, except for a few rules you must follow:

- You **cannot** simply instantiate an off-the-self PyTorch network. Instead, you must construct your network as a composition of existing PyTorch layers. In more concrete terms, you can use e.g. `torch.nn.Linear`, but you **cannot** use e.g. `torchvision.models.alexnet`.

- Justify every *design choice* you make. Design choices include network architecture, training hyperparameters, and, possibly, dataset preprocessing steps. You can either (i) start from the simplest convolutional network you can think of and add complexity one step at a time, while showing how each step gets you closer to the target ~60%, or (ii) start from a model that is already able to achieve the desired accuracy and show how, by removing some of its components, its performance drops (i.e. an *ablation study*). You can *show* your results/improvements however you want: training plots, console-printed values or tables, or whatever else your heart desires: the clearer, the better.

Don't be too concerned with your network performance: the ~60% is just to give you an idea of when to stop. Keep in mind that a thoroughly justified model with lower accuracy will be rewarded **more** points than a poorly experimentally validated model with higher accuracy.

In [13]:
def ncorrect(scores, y):
    y_hat = torch.argmax(scores, -1)
    return (y_hat == y).sum()

def accuracy(scores, y):
    correct = ncorrect(scores, y)
    return correct.true_divide(y.shape[0])

def train_loop(model, train_dl, epochs, opt, val_dl=None, verbose=False, label_smoothing=0):
    best_val_acc = 0
    best_params = []
    best_epoch = -1
    found = False
    for e in tqdm(range(epochs)):
        model.train()
        # Train
        train_loss = 0
        train_samples = 0
        train_acc = 0
        for train_data in train_dl:
            imgs = train_data[0].to(device)
            labels = train_data[1].to(device)
            scores = model(imgs)
            loss = F.cross_entropy(scores, labels,label_smoothing=label_smoothing)
            train_loss += loss.item() * imgs.shape[0]
            train_samples += imgs.shape[0]
            train_acc += ncorrect(scores, labels).item()

            opt.zero_grad()  # clear
            loss.backward()  # fill
            opt.step()       # use

        train_acc /= train_samples
        train_loss /= train_samples

        # Validation
        model.eval()
        with torch.no_grad():
            val_loss = 0
            val_samples = 0
            val_acc = 0
            if val_dl is not None:
                for val_data in val_dl:
                    imgs = val_data[0].to(device)
                    labels = val_data[1].to(device)
                    val_scores = model(imgs)
                    val_loss += F.cross_entropy(val_scores, labels).item() * imgs.shape[0]
                    val_samples += imgs.shape[0]
                    val_acc += ncorrect(val_scores, labels).item()
                val_acc /= val_samples
                val_loss /= val_samples

            if val_dl is None or val_acc > best_val_acc:
                best_val_acc = val_acc if val_dl is not None else 0
                best_params = model.state_dict()
                torch.save(best_params, "best_model.pth")
                best_epoch = e

        if verbose: #and e % 5 == 0:
            print(f"Epoch {e}: train loss {train_loss:.3f} - train acc {train_acc:.3f}" + ("" if val_dl is None else f" - valid loss {val_loss:.3f} - valid acc {val_acc:.3f}"))
            if val_acc >= 0.60:
                best_params = model.state_dict()
                torch.save(best_params, "A2.pth")
                best_epoch = e
                break

    if verbose and val_dl is not None:
        print(f"Best epoch {best_epoch}, best acc {best_val_acc}")

    return (best_val_acc, best_params, best_epoch)

# CNN

In [14]:
image_height, image_width, image_channels = 348,348,3 #224, 224, 3
num_classes = train_dataset.get_num_classes()

transform_2 = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(size=(image_height, image_width),antialias=True),
    #torchvision.transforms.Resize((image_height, image_width)),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean, std)
    #torchvision.transforms.Normalize((0.5, 0.4, 0.25), (0.23, 0.2, 0.17))
])

val_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((image_height, image_width)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean, std)
    #torchvision.transforms.transforms.Normalize((0.5, 0.4, 0.25), (0.23, 0.2, 0.17))
])
train_dataset = GroceryStoreDataset(split='train',transform=transform_2)
val_dataset = GroceryStoreDataset(split='val',transform=val_transforms)
#test_dataset = GroceryStoreDataset(split='test',transform=transform_2)

batch = 8
train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=False)

In [15]:
class ExtendedCNN(nn.Module):
    def __init__(self,channels=16, num_classes=num_classes,p_dropout=0):
        super(ExtendedCNN, self).__init__()
        # Define convolutional layers
        self.conv1 = nn.Conv2d(in_channels=image_channels, out_channels=channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=channels, out_channels=channels*2, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=channels*2, out_channels=channels*4, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=channels*4, out_channels=channels*8, kernel_size=3, stride=1, padding=1)

        # Calculate the size of the feature map after the convolutional layers
        def conv2d_size_out(size, kernel_size=3, stride=1, padding=1):
            return (size - kernel_size + 2 * padding) // stride + 1
        def conv2d_compute_size(conv_number,kernels,max_poolings):  #max pooling ex. [1,0,1] if 1 them there is a max_pool
          image_w = image_width
          for i in range(conv_number):
            image_w = conv2d_size_out(image_w, kernel_size=kernels[i])
            if max_poolings[i] == 1:
              image_w = image_w//2
          return image_w

        #convh = conv2d_size_out(conv2d_size_out(image_height, kernel_size=5, padding=1) // 2, kernel_size=3, padding=1) // 2
        image_w = conv2d_compute_size(4,[3,3,3,3],[1,1,1,1])

        linear_input_size = image_w * image_w * channels*8  # Adjusted based on the output channels of conv2

        # Define fully connected layers
        self.fc1 = nn.Linear(linear_input_size, 128)
        #self.drop1 = nn.Dropout(p_dropout)
        self.fc2 = nn.Linear(128, 64)
        #self.drop2 = nn.Dropout(p_dropout)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        # Apply convolutional layers with ReLU activation and max pooling
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        # Flatten the tensor
        x = x.view(x.size(0), -1)

        # Apply fully connected layers
        x = F.relu(self.fc1(x))
        #x = self.drop1(x)
        x = F.relu(self.fc2(x))
        #x = self.drop2(x)
        x = self.fc3(x)

        return x


In [ ]:
new_model = ExtendedCNN(
    num_classes = num_classes,
)
new_model.to(device)

#Training with: weight_decay=0.0, label_smoothing=0.5, lr=0.001, channels=16, batch_size=8

#optimizer = torch.optim.AdamW(new_model.parameters(), lr=0.001,weight_decay=0.01)
optimizer = torch.optim.Adam(new_model.parameters(), lr=0.001)

epochs = 80
label_smoothing = 0.1
best_val_acc, best_params, best_epoch = train_loop(
    new_model,
    train_loader,
    epochs,
    optimizer,
    val_loader,
    verbose=True,
    label_smoothing = label_smoothing,
)

  0%|          | 0/80 [00:00<?, ?it/s]

Epoch 0: train loss 3.243 - train acc 0.162 - valid loss 3.005 - valid acc 0.186
Epoch 1: train loss 2.826 - train acc 0.243 - valid loss 2.623 - valid acc 0.277
Epoch 2: train loss 2.589 - train acc 0.311 - valid loss 2.867 - valid acc 0.209
Epoch 3: train loss 2.363 - train acc 0.385 - valid loss 2.228 - valid acc 0.294
Epoch 4: train loss 2.179 - train acc 0.447 - valid loss 2.199 - valid acc 0.304
Epoch 5: train loss 2.089 - train acc 0.482 - valid loss 1.878 - valid acc 0.453
Epoch 6: train loss 1.953 - train acc 0.520 - valid loss 2.026 - valid acc 0.389
Epoch 7: train loss 1.878 - train acc 0.556 - valid loss 2.066 - valid acc 0.382
Epoch 8: train loss 1.832 - train acc 0.569 - valid loss 1.845 - valid acc 0.432
Epoch 9: train loss 1.753 - train acc 0.599 - valid loss 1.900 - valid acc 0.429
Epoch 10: train loss 1.714 - train acc 0.616 - valid loss 1.720 - valid acc 0.439
Epoch 11: train loss 1.665 - train acc 0.639 - valid loss 2.097 - valid acc 0.446
Epoch 12: train loss 1.628

In [20]:
from google.colab import drive
drive.mount('/content/drive')

new_model = ExtendedCNN(
    num_classes = num_classes,
)
new_model.to(device)

#Training with: weight_decay=0.0, label_smoothing=0.5, lr=0.001, channels=16, batch_size=8

#optimizer = torch.optim.AdamW(new_model.parameters(), lr=0.001,weight_decay=0.01)
optimizer = torch.optim.Adam(new_model.parameters(), lr=0.001)

# Chemin vers le fichier de poids sauvegardé
model_path = "/content/drive/MyDrive/AssignmentsIPCV/A2.pth"

# Charger les poids sauvegardés
new_model.load_state_dict(torch.load(model_path))

# Assurez-vous que le modèle est en mode évaluation
new_model.eval()

# Si vous avez besoin de tester le modèle ou de faire des prédictions, utilisez-le comme suit
# Exemple: prédiction sur des données de validation
with torch.no_grad():
    for val_data in val_loader:
        imgs = val_data[0].to(device)
        labels = val_data[1].to(device)
        outputs = new_model(imgs)
        # Traitez les outputs selon vos besoins

Mounted at /content/drive


# Gridsearch

In [22]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# # Montez Google Drive si nécessaire
# from google.colab import drive
# drive.mount('/content/drive')

# Assurez-vous que le dataset GroceryStoreDataset est disponible et bien structuré
# Utilisez la même transformation que celle utilisée pour le modèle précédent
transform_2 = transforms.Compose([
    transforms.RandomResizedCrop(size=(image_height, image_width), antialias=True),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_transforms = transforms.Compose([
    transforms.Resize((image_height, image_width)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset = GroceryStoreDataset(split='train', transform=transform_2)
val_dataset = GroceryStoreDataset(split='val', transform=val_transforms)

batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Load the pretrained model
resnet18 = torchvision.models.resnet18(weights=True)

# adapt to the dataset GroceryStoreDataset
num_classes = train_dataset.get_num_classes()
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)

# to use the gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = resnet18.to(device)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import torch.optim as optim

# Hyperparamater of Part A
learning_rate = 0.001
epochs = 80
label_smoothing = 0.1

# Définir l'optimiseur et la fonction de perte
optimizer = optim.Adam(resnet18.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)

# Function to train the model
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs, device):
    best_val_acc = 0
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        total_train = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_acc = train_correct / total_train

        model.eval()
        val_loss = 0
        val_correct = 0
        total_val = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_acc = val_correct / total_val

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = model.state_dict()
            torch.save(best_model_wts, "resnet18_best_model.pth")

        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss/total_train:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss/total_val:.4f}, Val Acc: {val_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

# Training
best_model = train_and_validate(resnet18, train_loader, val_loader, criterion, optimizer, epochs, device)


Epoch [1/80], Train Loss: 0.3831, Train Acc: 0.2367, Val Loss: 0.4182, Val Acc: 0.1926
Epoch [2/80], Train Loss: 0.3249, Train Acc: 0.3398, Val Loss: 0.3104, Val Acc: 0.3581
Epoch [3/80], Train Loss: 0.2940, Train Acc: 0.4170, Val Loss: 0.3269, Val Acc: 0.4595
Epoch [4/80], Train Loss: 0.2599, Train Acc: 0.5216, Val Loss: 0.3112, Val Acc: 0.4730
Epoch [5/80], Train Loss: 0.2412, Train Acc: 0.5667, Val Loss: 0.2846, Val Acc: 0.4966
Epoch [6/80], Train Loss: 0.2152, Train Acc: 0.6489, Val Loss: 0.2546, Val Acc: 0.5709
Epoch [7/80], Train Loss: 0.2091, Train Acc: 0.6682, Val Loss: 0.2653, Val Acc: 0.5811
Epoch [8/80], Train Loss: 0.1913, Train Acc: 0.7136, Val Loss: 0.2373, Val Acc: 0.6216
Epoch [9/80], Train Loss: 0.1895, Train Acc: 0.7333, Val Loss: 0.2517, Val Acc: 0.5777
Epoch [10/80], Train Loss: 0.1789, Train Acc: 0.7689, Val Loss: 0.2357, Val Acc: 0.6047
Epoch [11/80], Train Loss: 0.1696, Train Acc: 0.7909, Val Loss: 0.2243, Val Acc: 0.6318
Epoch [12/80], Train Loss: 0.1652, Train 

In [16]:
# weight_decays = [0.0,0.01,0.001]
# label_smoothings = [0.0, 0.05, 0.1, 0.15, 0.2]
# lrs = [0.001]#, 0.0001]
# channels = [16]
# batches = [8]
# #p_dropout = [0.1,0.3,0.5]

# train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=True)
# #test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=False)

# epochs = 80
# test_iteration = 1
# total_test = len(weight_decays)*len(label_smoothings)*len(lrs)*len(channels)*len(batches)#*len(p_dropout)
# #Training with: weight_decay=0.0, label_smoothing=0.5, lr=0.001, channels=16, batch_size=8

# for weight_decay in weight_decays:
#     for label_smoothing in label_smoothings:
#         for lr in lrs:
#             for channel in channels:
#                 for batch in batches:
#                     model = ExtendedCNN(channels=channel, num_classes=num_classes,p_dropout=0)
#                     model.to(device)

#                     optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

#                     print(f"Training with: weight_decay={weight_decay}, label_smoothing={label_smoothing}, lr={lr}, channels={channel}, batch_size={batch},test={test_iteration}/{total_test}")
#                     test_iteration+=1
#                     best_val_acc, best_params, best_epoch, found = train_loop(
#                         model,
#                         train_loader,
#                         epochs,
#                         optimizer,
#                         val_loader,
#                         verbose=True,
#                         label_smoothing = label_smoothing,
#                     )